In [2]:
import pandas as pd
import os

In [3]:
MENU_LATEST =  os.path.dirname(os.getcwd())+'/master_menu/menu.csv'
NUTRITION_SUPERMASTER_LATEST = 'super_master_nutrition/supermaster.csv'
df = pd.read_csv(MENU_LATEST)
df_n = pd.read_csv(NUTRITION_SUPERMASTER_LATEST,converters={'cafeteria_id': lambda x: str(x), 'category': lambda x: str(x)})

df.head(3)

,group,timestamp,name,items,week_num,items_recipe_num,pk
0,afterlunch,2022-03-06 14:30:00Z,"Afterlunch on Sunday, March 06, 2022",[],9,"['102407', '44726.1', '14920', '82258.7']",1000
1,breakfast,2022-03-06 07:30:00Z,"Breakfast on Sunday, March 06, 2022","[1030, 1012, 1022, 1037, 1025]",9,"['2032.23', '18451.22', '616.22', '1125.10', '...",1001
2,dinner,2022-03-06 16:30:00Z,"Dinner on Sunday, March 06, 2022","[1126, 1145, 1147, 1135, 1149, 1169, 1188, 117...",9,"['102098', '4490', '2402', '103772', '33992.8'...",1002


In [4]:
df['items'] = df['items'].apply(lambda x:eval(x))
df['items_recipe_num'] = df['items_recipe_num'].apply(lambda x: eval(x))

In [5]:
DELTA=[]
for index, row in df.iterrows():
    DELTA.append(len(row['items_recipe_num'])-len(row['items']))

In [6]:
row_index = []
for index, row in df.iterrows():
    if len(row['items_recipe_num'])-len(row['items'])>0:
    #if len(row['items']) == 0 and len(row['items_recipe_num'])>0:
        row_index.append(index)

In [7]:
print(row_index)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119]


In [11]:
MISSING_DISHES = []
BAD_UNITS_COUNT = []
BAD_UNITS=[]
for i in row_index:
    missing_dishes = df.loc[i, 'items_recipe_num']
    u = []
    for h in missing_dishes:
        mm = df_n.loc[df_n[df_n['cafeteria_id'] == h].index, 'portion_volume']
        if mm.empty is True:
            u.append('')
        else:
            try:
                float(mm)
            except:
                u.append(mm.values.tolist()[0])
    if u.count('') == len(u):
        MISSING_DISHES.append(u.count(''))
        BAD_UNITS_COUNT.append(0)
        BAD_UNITS.append(None)
    else:
        MISSING_DISHES.append(u.count(''))
        BAD_UNITS_COUNT.append(len(u)-u.count(''))  
        BAD_UNITS.append(list(filter(('').__ne__, u)))

In [9]:
UNITS_COUNT = [None for i in range(len(df))]
UNITS = [None for i in range(len(df))]
DISHES = [None for i in range(len(df))]
k_unit_c = list(zip(row_index, BAD_UNITS_COUNT))
k_unit = list(zip(row_index, BAD_UNITS))
k_dish = list(zip(row_index, MISSING_DISHES))
for kk in k_unit:
    UNITS[kk[0]] = kk[1]
    
for kk in k_unit_c:
    UNITS_COUNT[kk[0]] = kk[1]
    
for kk in k_dish:
    DISHES[kk[0]] = kk[1]

df['num_missing_Dish'] = DELTA
df['num_bad_units'] = UNITS_COUNT
df['num_missing_FDA'] = DISHES
df['bad_units'] = UNITS

In [10]:
df.to_csv('Menu_Study_Analysis.csv')